In [1]:
# Copyright (c) ContextualFairness contributors.
# Licensed under the MIT License.

In [2]:
# Make sure we can import contextualfairness
import sys

sys.path.append("../")

In [3]:
import pandas as pd

from contextualfairness import (
    contextual_fairness_score,
    BinaryClassificationEqualityNorm,
    RankNorm,
)

In [4]:
# Setup fake data
ages = ["young", "young", "old", "young", "old"]
y_true = [1, 0, 0, 1, 0]

d = {
    "income": [50, 80, 30, 100, 30],
    "age": ages,
    "sex": ["male", "female", "male", "female", "male"],
    "y true": y_true,
}

X = pd.DataFrame(data=d)

In [5]:
# Define fake predictions and according probabilities
y_pred = [1, 0, 1, 0, 1]
y_pred_probas = [0.5, 0.2, 0.6, 0.4, 0.5]

In [6]:
# Define norm function
def richer_is_better(x):
    return x["income"]

In [7]:
norms = [BinaryClassificationEqualityNorm(), RankNorm(richer_is_better)]

result = contextual_fairness_score(
    norms=norms,
    X=X,
    y_pred=y_pred,
    y_pred_probas=y_pred_probas,
    weights=[0.4, 0.6],
)

In [8]:
result.df

,income,age,sex,y true,Equality,richer_is_better,total
0,50,young,male,1,0.00,0.05,0.05
1,80,young,female,0,0.25,0.15,0.40
2,30,old,male,0,0.00,0.00,0.00
3,100,young,female,1,0.25,0.15,0.40
4,30,old,male,0,0.00,0.00,0.00


In [9]:
result.total_score()

np.float64(0.8500000000000001)

In [10]:
result.group_scores(["sex"])

{'sex=female': {'data': 1    0.4
  3    0.4
  Name: total, dtype: float64,
  'score': np.float64(0.8)},
 'sex=male': {'data': 0    0.05
  2    0.00
  4    0.00
  Name: total, dtype: float64,
  'score': np.float64(0.05)}}

In [11]:
result.group_scores(["sex", "age"], scaled=True)

{'sex=female;age=old': {'data': Series([], Name: total, dtype: float64),
  'score': np.float64(0.0)},
 'sex=female;age=young': {'data': 1    0.377778
  3    0.377778
  Name: total, dtype: float64,
  'score': np.float64(0.7555555555555556)},
 'sex=male;age=old': {'data': 2    0.0
  4    0.0
  Name: total, dtype: float64,
  'score': np.float64(0.0)},
 'sex=male;age=young': {'data': 0    0.094444
  Name: total, dtype: float64,
  'score': np.float64(0.09444444444444446)}}